In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import gtsam
from gtsam.utils import plot
import matplotlib.pyplot as plt
import rosbag

In [3]:
from sensor_msgs.msg import LaserScan
def preprocess_measurement(laser_msg: LaserScan, min_range=0, max_range=np.inf):
    """Extracts ranges from the LaserScan message and converts into
    a 2*N homogenous matrix of points.

    Args:
        laser_msg: The LaserScan message from the A1.
    Returns:
        scan: A numpy array of shape (2, N), where N is the number of 2D points.
    """
    scan = np.zeros((2, len(laser_msg.ranges)), dtype=float)
    # Compute the 2D point where the angle starts at +90 deg, and
    # each range is spaced out in equal increments.
    for i, distance in enumerate(laser_msg.ranges):
        if min_range < distance < max_range:
            scan[0][i] = distance * \
                np.cos(laser_msg.angle_min + i*laser_msg.angle_increment)
            scan[1][i] = distance * \
                np.sin(laser_msg.angle_min + i*laser_msg.angle_increment)
    # Remove all values in the array that are still 0.
    mask = scan[0] != 0
    scan = scan[:, mask]

    return scan

In [15]:
traj_num = 4
df = pd.read_csv(f"/home/jerredchen/A1_trajs_06-04-2022/traj{traj_num}.csv")

for i in range(df.shape[1]):
    name1 = df.iloc[0, i]
    name2 = df.iloc[2, i]
    name3 = df.iloc[3, i] if str(df.iloc[3, i]) != 'nan' else ''
    df.iloc[3, i] = f"{name1} {name2} {name3}".strip()
df.iloc[3, 0] = 'Frame'
df.iloc[3, 1] = 'Time'
renamed_cols_map = {}
for i, col in enumerate(df.columns):
    renamed_cols_map[col] = df.iloc[3, i]
df = df.rename(columns=renamed_cols_map)
df = df.drop([0, 1, 2, 3])
df = df.reset_index(drop=True)

/home/jerredchen/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning:

Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245) have mixed

In [80]:
"""
1. Plot the mocap trajectory at all time steps, corrected to align properly.
2. Find a specific point on the trajectory that matches for mocap and slam.
3. Find the associated timestamp for each, and assume that they match perfectly.
4. For every other slam pose, find the closest mocap timestamp that aligns with the slam timestamp.
"""

'\n1. Plot the mocap trajectory at all time steps, corrected to align properly.\n2. Find a specific point on the trajectory that matches for mocap and slam.\n3. Find the associated timestamp for each, and assume that they match perfectly.\n4. For every other slam pose, find the closest mocap timestamp that aligns with the slam timestamp.\n'

In [7]:
cwTi = gtsam.Pose3()
bTlidar = gtsam.Pose3(gtsam.Rot3(), np.array([0.1385, -0.016, 0.0744]))
lidarTbase = gtsam.Pose3(gtsam.Rot3(), np.array([0.10944, 0.01562, 0.1244])).inverse()
bTbase = bTlidar.compose(lidarTbase)
baseTlidar = gtsam.Pose3(gtsam.Rot3(), np.array([0.10944, 0.01562, 0.1244]))
rot = np.array([
    [0, 0, 1],
    [1, 0, 0],
    [0, 1, 0]
]).T
rwTcw = gtsam.Pose3(gtsam.Rot3(rot), np.zeros((3,)))
prev_pose = gtsam.Pose3()
trajectory = []
ind = 0

mocap_time_point = 0
mocap_times = []
mocap_poses = []

for i in range(0, df.shape[0]):
# for i in range(df.shape[0]):

    # Obtain the rotation and translation from the MOCAP.
    qw = float(df['Rigid Body 1 Rotation W'][i])
    qx = float(df['Rigid Body 1 Rotation X'][i])
    qy = float(df['Rigid Body 1 Rotation Y'][i])
    qz = float(df['Rigid Body 1 Rotation Z'][i])
    rotation = gtsam.Rot3.Quaternion(qw, qx, qy, qz)
    # rotation = gtsam.Rot3()
    tx = float(df['Rigid Body 1 Position X'][i])
    ty = float(df['Rigid Body 1 Position Y'][i])
    tz = float(df['Rigid Body 1 Position Z'][i])
    translation = gtsam.Point3(tx, ty, tz)

    # Calculate the pose in the camera's world frame.
    if len(mocap_poses) == 0:
        cwTi = gtsam.Pose3(rotation, translation)
        time = float(df['Time'][i])
        mocap_times.append(time)
        mocap_poses.append(gtsam.Pose3())
        continue
    cwTj = gtsam.Pose3(rotation, translation)

    # Calculate the relative pose between pose i and j.
    iTj = cwTi.between(cwTj)
    iTj = rwTcw.inverse().compose(iTj).compose(rwTcw)
    iTj = bTbase.inverse().compose(iTj).compose(bTbase)

    # Calculate wTj and transform into the robot frame.
    rwTj = prev_pose.compose(iTj)

    time = float(df['Time'][i])
    if np.allclose([0.9440, -1.9036, 0.1651], rwTj.translation(), atol=1e-4):
        mocap_time_point = time        
        print(f"{mocap_time_point=}")
    mocap_times.append(time)
    mocap_poses.append(rwTj)
    cwTi = cwTj
    prev_pose = rwTj

mocap_time_point=40.058333


In [12]:
a1_slam_vals = np.loadtxt(f'/home/jerredchen/borglab/evaluation-evo/traj{traj_num}_a1_slam.txt')
a1_point_index = 0
a1_time_point = 0
for i in range(a1_slam_vals.shape[0]):
    translation = a1_slam_vals[i, 1:4]
    if np.allclose([0.8641, -1.8831, 0], translation, atol=1e-3):
        a1_time_point = a1_slam_vals[i, 0]
        a1_point_index = i
        print(f"{a1_time_point=}")
# Iterate through all a1_slam poses.
mocap_trajectory = []
mocap_time_start = 0
for j in range(a1_slam_vals.shape[0]):
    time_gap = a1_slam_vals[j, 0] - a1_time_point
    idx = (np.abs(mocap_times - (mocap_time_point + time_gap))).argmin()
    pose = mocap_poses[idx]
    time = mocap_times[idx]
    if j == 0:
        mocap_time_start = time
    mocap_trajectory.append((time - mocap_time_start, pose))

a1_time_point=7.84746527671814


In [20]:
traj_num = 5
a1_slam_vals = np.loadtxt(f'/home/jerredchen/borglab/evaluation-evo/traj{traj_num}_a1_slam.txt')
carto_vals = np.loadtxt(f'/home/jerredchen/borglab/evaluation-evo/traj{traj_num}_cartographer.txt')
mocap_vals = np.loadtxt(f'/home/jerredchen/borglab/evaluation-evo/traj{traj_num}_mocap.txt')

x = mocap_vals[:,1]
y = mocap_vals[:,2]
z = np.zeros(mocap_vals.shape[0])
fig = px.scatter_3d(x=x, y=y, z=z, range_x=[-0.1, 2], range_y=[-2.5, 0], range_z=[-0.1, 1.0])

x = a1_slam_vals[:,1]
y = a1_slam_vals[:,2]
z = a1_slam_vals[:,3]
fig.add_scatter3d(x=x, y=y, z=z, name="a1_slam")

x = carto_vals[:, 1]
y = carto_vals[:, 2]
z = carto_vals[:, 3]
fig.add_scatter3d(x=x, y=y, z=z, name="cartographer")
fig.update_traces(marker_size=1.0)
fig.show()

In [14]:
with open(f'traj{traj_num}_mocap.txt', 'w') as f:
  for ts, pose in mocap_trajectory:
    tx, ty, tz = pose.translation()
    rotation = pose.rotation().ypr()
    projected_rotation = gtsam.Rot3.Ypr(rotation[0], 0, 0)
    qw, qx, qy, qz = projected_rotation.quaternion()
    f.write(f"{ts} {tx} {ty} 0.0 {qx} {qy} {qz} {qw}")
    f.write('\n')

In [2]:
B_quality_points = 3 * (4 + 3 + 3 + 3 +3)
A_quality_points = 4 * (21 + 48)
(A_quality_points + B_quality_points) / (4 * (21 + 48 + 16)) * 4

3.8117647058823527